In [34]:
import pandas as pd
import numpy as np
import pickle

import datetime as dt
import matplotlib.pyplot as plt
from matplotlib import style

import pandas_datareader.data as web

import matplotlib.dates as mdate
# %matplotlib notebook # interactive navigation bar 


from collections import Counter
from sklearn import svm, model_selection, neighbors
from sklearn.ensemble import VotingClassifier, RandomForestClassifier

style.use('ggplot')

In [7]:
def process_data_for_labels(ticker):
    hm_days = 7
    df = pd.read_csv('sp500_joined_closes.csv', index_col = 0)
    tickers = df.columns.values.tolist()
    df.fillna(0,inplace=True)
    
    for i in range(1,hm_days+1):
        df['{}_{}d'.format(ticker,i)] = (df[ticker].shift(-i) - df[ticker] )/ df[ticker]
    df.fillna(0, inplace=True)
    
    return tickers, df


tickers, df = process_data_for_labels('AAPL')
print(len(tickers))
df.head(2)

49


,MMM,ABT,ABBV,ABMD,ACN,ATVI,ADBE,AMD,AAP,AES,...,APA,AIV,AAPL,AAPL_1d,AAPL_2d,AAPL_3d,AAPL_4d,AAPL_5d,AAPL_6d,AAPL_7d
Date,,,,,,,,,,,,,,,,,,,,,
1999-12-31,27.323933,6.810734,0.0,18.375,0.0,1.143539,16.693562,14.46875,0.0,29.469620,...,12.383977,14.637461,3.187337,0.088754,-0.003039,0.011550,-0.075988,-0.032219,-0.049240,-0.097872
2000-01-03,26.346825,6.564564,0.0,18.250,0.0,1.227554,16.274673,15.50000,0.0,28.582567,...,11.441033,13.971079,3.470226,-0.084310,-0.070910,-0.151312,-0.111111,-0.126745,-0.171413,-0.221106


In [11]:
def buy_sell_hold(*args):
    cols = [c for c in args]
    requirement = 0.02
    for col in cols :
        if col > requirement:
            return 1
        if col < -requirement:
            return -1
    return 0


In [27]:


def extract_featuresets(ticker):
    tickers , df = process_data_for_labels(ticker)
    # 7일 이내 한 번이라도 매수/매도 신호가 생기면 시그널 발생
    # input : 7일 데이터, output : signal
    df['{}_target'.format(ticker)] = list(map(buy_sell_hold, 
                                             df['{}_1d'.format(ticker)],
                                              df['{}_2d'.format(ticker)],
                                              df['{}_3d'.format(ticker)],
                                              df['{}_4d'.format(ticker)],
                                              df['{}_5d'.format(ticker)],
                                              df['{}_6d'.format(ticker)],
                                              df['{}_7d'.format(ticker)]
                                             )) 
    vals = df['{}_target'.format(ticker)].values.tolist()
    str_vals = [str(i) for i in vals]
    print('Data spread:', Counter(str_vals))
    
    df.fillna(0,inplace=True)
    df = df.replace([np.inf, -np.inf], np.nan)
    df.dropna(inplace=True) 
    
    df_vals = df[[ticker for ticker in tickers]].pct_change()
    df_vals = df_vals.replace([np.inf, -np.inf], np.nan)
    df_vals.fillna(0,inplace=True)
    
    X= df_vals.values # daily return rate
    y = df['{}_target'.format(ticker)].values # daily signal 
    return X,y, df

daily_return_rate, daily_signal, df_with_signal =  extract_featuresets('AAPL')
    
    
    

Data spread: Counter({'1': 2494, '-1': 2039, '0': 499})


In [39]:
def do_ml(ticker):
    X,y, df = extract_featuresets(ticker)
    X_train,X_test,y_train,y_test  = model_selection.train_test_split(X,y, test_size = .25)
    
#     clf = neighbors.KNeighborsClassifier()
    clf = VotingClassifier([
        ('lsvc', svm.LinearSVC()),
        ('knn',neighbors.KNeighborsClassifier()),
        ('rfor',RandomForestClassifier())
    ])

    clf.fit(X_train, y_train) 
    accuracy = clf.score(X_test, y_test) # model accuracy
    predictions = clf.predict(X_test) # predicted classes
    print('Accuracy: ', accuracy)
    print('Predicted spread: ', Counter(predictions)) # predicted results
    return accuracy

do_ml('AAPL')

Data spread: Counter({'1': 2494, '-1': 2039, '0': 499})
Accuracy:  0.4960254372019078
Predicted spread:  Counter({1: 992, -1: 265, 0: 1})


0.4960254372019078